# Supervisor (Polyglot)

In [ ]:
#!import ../nbs/Testing.dib

In [ ]:
#r @"../../../../../../../.nuget/packages/fsharp.control.asyncseq/3.2.1/lib/netstandard2.1/FSharp.Control.AsyncSeq.dll"
#r @"../../../../../../../.nuget/packages/system.reactive/5.0.0/lib/net5.0/System.Reactive.dll"
#r @"../../../../../../../.nuget/packages/system.reactive.linq/6.0.1-preview.1/lib/netstandard2.0/System.Reactive.Linq.dll"

#r @"../../../../../../../.nuget/packages/asyncio/0.1.69/lib/netstandard2.0/AsyncIO.dll"
#r @"../../../../../../../.nuget/packages/netmq/4.0.1.12/lib/netstandard2.1/NetMQ.dll"

In [ ]:
#!import ../nbs/Common.fs
#!import ../nbs/Async.fs
#!import ../nbs/AsyncSeq.fs
#!import ../nbs/Networking.fs
#!import ../nbs/Runtime.fs
#!import ../nbs/FileSystem.fs

In [ ]:
open Common
open FileSystem

In [ ]:
let inline sendJson (port : int) (json : string) = async {
    let! portOpen = Networking.testPortOpen port
    if portOpen then
        // use runtime = new NetMQ.NetMQRuntime ()
        use request = new NetMQ.Sockets.RequestSocket ()
        request.Connect $"tcp://127.0.0.1:{port}"

        let msg = NetMQ.NetMQMessage ()
        msg.Append json

        NetMQ.OutgoingSocketExtensions.SendMultipartMessage (request, msg)
        let result = NetMQ.ReceivingSocketExtensions.ReceiveMultipartMessage (request, 10)
        // let! result = NetMQ.AsyncReceiveExtensions.ReceiveMultipartMessageAsync (request, 10) |> Async.AwaitTask
        let result = result |> Seq.toList |> List.map (fun x -> x.ConvertToString ())
        trace Debug (fun () -> $"sendJson / port: {port} / json: {json} / result: {result}") getLocals
        return true
    else
        trace Debug (fun () -> "sendJson / error: port not open") getLocals
        return false
}

In [ ]:
let inline sendObj port obj =
    let json = System.Text.Json.JsonSerializer.Serialize obj
    sendJson port json

In [ ]:
let inline compile code = async {
    let tempDir = FileSystem.createTempDirectory ()

    let stream, disposable = FileSystem.watchDirectory true tempDir

    let mainPath = tempDir </> "main.spi"
    let spiprojPath = tempDir </> "package.spiproj"

    let inline writeCode path code =
        System.IO.File.WriteAllTextAsync (path, code) |> Async.AwaitTask

    do! code |> writeCode mainPath
    do! "packages:
    |core-
modules:
    main" |> writeCode spiprojPath

    let port = 13805

    let! ct = Async.CancellationToken
    let compiler = MailboxProcessor.Start (fun inbox -> async {
        let! availablePort = Networking.getAvailablePort (Some 60) port
        if availablePort <> port
        then inbox.Post ()
        else
            let compilerPath =
                "../../deps/The-Spiral-Language/The Spiral Language 2/artifacts/bin/The Spiral Language 2/release"
                |> Path.GetFullPath

            let dllPath = compilerPath </> "Spiral.dll"
            // let commandsPath = compilerPath </> "compiler/supervisor/commands"

            let! exitCode, result =
                Runtime.executeWithOptionsAsync
                    {
                        Command = $@"dotnet ""{dllPath}"" port={port}"
                        CancellationToken = None
                        WorkingDirectory = None
                        OnLine = Some <| fun { Line = line } -> async {
                            if line |> String.contains $"Server bound to: tcp://*:{port}"
                            then inbox.Post ()
                        }
                    }
            trace Debug (fun () -> $"startSupervisor / exitCode: {exitCode} / result: {result}") getLocals
    }, ct)

    do! compiler.Receive ()

    let inline getFileUri path =
        $"file:///{path |> String.trimStart [| '/' |]}"


    let fileOpenObj = {| FileOpen = {| uri = mainPath |> getFileUri; spiText = code |} |}
    let! fileOpenResult = fileOpenObj |> sendObj port

    let buildFileObj = {| BuildFile = {| uri = mainPath |> getFileUri; backend = "Fsharp" |} |}
    let! buildFileResult = buildFileObj |> sendObj port

    let! fsxContent =
        stream
        |> FSharp.Control.AsyncSeq.choose (function
            | _, FileSystemChange.Changed ("main.fsx", Some content) -> Some content
            | _ -> None
        )
        |> FSharp.Control.AsyncSeq.tryFirst

    disposable.Dispose ()

    return fsxContent |> Option.defaultValue "" |> String.replace "\r\n" "\n"
}

In [ ]:
//// test


"inl app () =
    0i32

inl main () =
    app
    |> dyn
    |> ignore
"
|> compile
|> Async.runWithTimeout 8000
|> _equal (Some "let rec closure0 () () : int32 =
    0
let v0 : (unit -> int32) = closure0()
()
")

00:40:47 #1 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230731-0040-4733-3367-3e8531f6a2d1 / result: { CreationTime = 2023-07-31 12:40:47 AM
  Exists = true }
00:40:47 #2 [Debug] runWithTimeoutAsync / timeout: 60
00:40:47 #3 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805"
  WorkingDirectory = None
  CancellationToken = None
  OnLine = Some <fun:it@12-220> }
00:40:48 #4 [Debug]  7204: Server bound to: tcp://*:13805 & tcp://*:13806
00:40:48 #5 [Debug]  7204: pwd: C:\home\git\polyglot\apps\spiral
00:40:48 #6 [Debug]  7204: dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:40:48 #7 [Debug] sendJson / port: 13805 / json: {"FileOpen":{"spiText":"inl app () =\n    0i32\n\ninl main () =\n    app\n    |\u003E dyn\n    |\u003E ignore\

In [ ]:
//// test


"""inl write_line ~obj : () =
    $"System.Console.WriteLine !obj"

inl app () =
    write_line "error"
    1i32

inl main () =
    app
    |> dyn
    |> ignore
"""
|> compile
|> Async.runWithTimeout 8000
|> _equal (Some """let rec closure0 () () : int32 =
    let v0 : string = "error"
    System.Console.WriteLine v0
    1
let v0 : (unit -> int32) = closure0()
()
""")

00:40:53 #11 [Debug] createTempDirectory / tempFolder: C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20230731-0040-5345-4579-4d9225d13016 / result: { CreationTime = 2023-07-31 12:40:53 AM
  Exists = true }
00:40:53 #12 [Debug] runWithTimeoutAsync / timeout: 60
00:40:53 #13 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" port=13805"
  WorkingDirectory = None
  CancellationToken = None
  OnLine = Some <fun:it@16-556> }
00:40:54 #14 [Debug]  5276: Server bound to: tcp://*:13805 & tcp://*:13806
00:40:54 #15 [Debug]  5276: pwd: C:\home\git\polyglot\apps\spiral
00:40:54 #16 [Debug]  5276: dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:40:54 #17 [Debug] sendJson / port: 13805 / json: {"FileOpen":{"spiText":"inl write_line ~obj : () =\n    $\u0022System.Console.WriteLine !obj\u0022\n\ni